Welcome to my EDA, prior to making a Convolutional Neural Network (CNN). CNN are commonly used for classification or binary problems. 
We will use this exploration to determine how we are going to build our model, by testing accuracy and build. Then when we are happy we will create the model in a py file, and make it ready to deploy onto a flask based API.

In this first section we will explore and process our inputs by doing the following:
1) Download the dataset and visualise the images ( CIFAR-10 )
2) Change the label to one hot encodings ( Create a binary classification columns)
3) Scale the image pixel values to take between 0 and 1 ( So our image can be processed by a numerically based model

Details about the CIFAR-10 dataset:
1) Images ( 32*32 pixels) (This is the scale of the images)
2) labels 10 possible identifier labels ( This can be amended dependant on what you are analysing. In this case we have ten different types of animals we can identify)
3) Dataset size is  60k images, (We will split this into 50k training and 10k testing)

In [ ]:
# Lets begin by importing our dataset.
from keras.datasets imports cifar10
# The dataset has already been split at source:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

However, if your receive images that have not yet been split you could use the following:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.17, random_state=50)

Test size is 0.17 as 50k(training size) of 60k images is 0.83

In [ ]:
# Just to check our imported data has split correctly. Lets run the following:
print('x_train shape:', x_train.shape)

The result above tells us that our training data is made up of 50k images, which are 32 pixels in height, 32 pixels in width and 3 pixels in depth

In [ ]:
# Lets see what the shape of the label array is.
print('y_train shape:', y_train.shape)

In [ ]:
# We can also investigate what an image looks like, when it is broken down into pixels
print(x_train[0])

In [ ]:
# Above we can see the image result is almost a tensore or array of numbers.
# However, this does not tell use very much.
# Lets use matplotlib.pyplot to visualise this in a better way

import matplotlib.pyplot as plt
%matplotlib inline

img = plt.imshow(x_train[0])

In [ ]:
# It is very hard to make out what the image is due to pixelation.
# Lets see what the training data classifies this image as

print('the label is:', y_train[0])

If we are building a predictor which identifies is we will experience above or below median we can set only used 2 labels, 1(above) or 0(below), this is because there was only two possible outputs. Furthermore it is important to consider that many numerical problems, will have better approaches that forcing a Neural Network. Such as, Random Forests, Decision Trees, Regression.

However, image input recognition is a multi classing problem, where there is 10 possible outputs in this case. As a result our outputs possibility will be 10 neurons or classifications.

In order to solve the multi class issue and respect that the CNN needs numerical factors. We will initiate a one hot encoding feature
One hot encoding, will take the approach that if the image belongs to first class, all relevant rows of this column will be assigned a number of 1, the others 0

If the image belongs to the second class, all relevant rows of this second columns will be set to 1 and so on.

In [ ]:
# Lets initiate our one hot encoding using keras
import keras
y_train_one_hot = keras.utils.to_categorical(y_train, 10)
y_test_one_hot = keras.utils.to_categorial(y_test, 10)

We need to talk about what just happened above. The number 10, symbolizes the 10 different possible classifications an image can receive.
However, be aware that columns are counted as 0-9 in python.


At this point we have set our class classification as one hot encoding so we can produce results from our CNN.
Now we want to process our image(x)

A common approach to this, is to allow the numbers to be between 0 and 1, this will aid in the training of our neural network
Since our pixel values already take the values between 0 and 255 (32x32), we simply divide by 255

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255

Checkpoint! Up until this point, we have:
1) Downloaded the dataset and visualised the images using matplotlib
2) Changed the y labels to one hot encodings. This makes them numerical and able to pass into our CNN
3) scaled the image pixel value to take between 0 and 1. By dividing by the pixel size we have in this dataset

BUILDING AND TRAINING OUR CONVOLUTIONAL NEURAL NETWORK

We have arrived at the point where we will define our model.

The CNN architecture, which we will be building is as follows:

1) we input our image

2) we have our first Conv layer ( filter = 3x3, stride = 1,Depth = 32)

3) we have our second Conv layer ( filter = 3x3, stride = 1,Depth = 32)

4) we have a Max pool Layer ( filter = 2x2, stride = 2)

5) we have a dropout layer (probability:0.25)

6) we have our third Conv layer ( filter = 3x3, stride = 1,Depth = 64)

7) we have our fourth Conv layer ( filter = 3x3, stride = 1,Depth = 64)

8) we have a second Max pool Layer ( filter = 2x2, stride = 2)

9) we have a second dropout layer (probability:0.25)

10) we then have a FC Layer ( Neurons = 512)

11) we have a third dropout layer (probability:0.5)

12) we then have a second FC Layer ( Neurons =10)

13) we then have a softmax Layer

14) finally we have our classification results

What is cool about this, is that we can amend and play with the model set up ourselves, to see if we can improve our results

The softmax layer (13), transforms the output of the layer into probability distributions, which is what we want for our classification problem

What we have not specified in the model breakdown above is padding. For now, we will zero pad our layer. So output width and height is the same as the input width and height.

This is called same padding, for a 3x3 filter, to achieve the same width and height, we have pad with a border of 1

Finally we will use the ReLU activation for all our layers, except the last(which uses a softmax)

To code the above breakdown into a functional model, we will use the Keras sequential model. However, since there is many layers in our model, we will do this as shown below

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [ ]:
## Next we will call an 'empty' sequential model
model =  Sequential()

In [ ]:
## Now the process, is to add a layer by layer to this empty model one at a time:
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32,32,3)))  # first layer, we did not specify stride, as this is default as 1, we specify if we want it to be different
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))   # we do not need to put input layer twice, since it can infer from previous input
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())    # At this point neurons are like  acube, we need to flatten them into one row using this function
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
#Above we have manually input and created our model. We can now proceed to start filling in the best numbers
# We will compile the model with the settings below:

model.compile(loss='categorical_crossentropy',
              optimizer='adam',           # adam is a sgd, with a few modifications so it trains better
              metrics=['accuracy'])

In [ ]:
## Now we can start training our model
## If you notice we set up our validation split at 0.2. This is a shortcut that means we did not need to split our dataset up at the beginning,
## instead we specify how much of our dataset will be used as a validation set.

Classifier = model.fit(x_train, y_train_one_hot,
                 batch_size=32, epochs=5,
                 validation_split=0.2)

In [ ]:
# Now that we have completed our model training, we can visualize the model training and validation loss over the number of epochs

plt.plot(Classifier.history['loss'])
plt.plot(Classifier.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()


In [ ]:
## Lets also have a look at the accuracy of what we created:

plt.plot(Classifier.history['accuracy'])
plt.plot(Classifier.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

At this point we can see the valuation accuracy is a lot lower than the test accuracy. This could be due to underfitting, or the have created our model. At this point we should investigate:

1) hyper parameters
2) number of epochs
3) batch size

Once we are happy we can then run our test set to see how well it performs on unseen data. (Note: as this is a demonstration of how to create such a model. At this point in time I have not spent much time tweaking it)

In [ ]:
# Lets evaluate how the model does on the test data.
model.evaluate(x_test, y_test_one_hot)[1]

Not bad, we have an accuracy of 77%


If you are planning to stay in the Jupyter notebook you can use the following to save your model. However, in a more professional setting it is less likely that you will keep them as such. Nonetheless:

We can now save our trained model, since it took so long to train by using the following code:

model.save('my_cifar10_model.h5')

if you want to access this model again in the future, we can use this line of code:

from keras.models import load_model
model= load_model('my_cifar10_model.h5')

Again, this part, assumes you will test your model in a jupyter notebook setting:

TESTING OUR MODEL

WE CAN IMPORT A IMAGE INTO OUR DIRECTORY
THEN WE CAN IMPORT IT USING THE FOLLOWING CODE:
MY_IMAGE = PLT.IMREAD("CAT.JPG")

WE THEN NEED TO RESIZE THE IMAGE, WE CAN DO THIS MANUALLY OR, "IMPORT SCIKIT-IMAGE" INSIDE THE ANAZONDA NAVIGATOR
ONCE WE HAVE COMPLETED THAT, GO BACK TO JUPYTER NOTEBOOK AND START USING THE NEEDED FUNCTIONS:

FROM SKIMAGE.TRANSFORM IMPORT RESIZE
MY_IMAGE_RESIZED =  RESIZE(MY_IMAGE, (32,32,3))


WE CAN VISUALISE OUR IMAGE TO TEST IT WORKED USING:
#/ IMG = PLT.IMSHOW(MY_IMAGE_RESIZED)

THEN WE CAN VIEW WHAT OUR TRAINED MODEL WOULD OUTPUT BASED ON THEIS PICTURE:
IMPORT NUMPY AS NP
PROBABILITIES = MODEL.PREDICT(NP.ARRACY([MY_IMAGE_RESIZED,]))

WE CAN THEN VIEW IT RUNNING THE CODE:
PROBABILITIES

THE PROBABILITIES ARE OUTPUT IN ORDER FROM 0-10

##### Next steps

Once you are happy with your data exploration and the model accuracy. The next step is creating the model in a .py file. Then being able to deploy it onto a development server using Flask. You will find the remaining files in the directory.